In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

import numpy as np
import time

In [4]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])
MM_WALLET = WalletConfig("mm", "mm")
TRADER_1_WALLET = WalletConfig("T1", "pin")
TRADER_2_WALLET = WalletConfig("T2", "pin")
TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TERMINATE_WALLET]

In [5]:
vega = VegaServiceNull(run_with_console=True, use_full_vega_wallet=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/67/mjxp58z56yj83_1x0372gkwr0000gn/T/tmpmot1k5hm
INFO:vega_sim.null_service:Launching GraphQL node at port 52231
INFO:vega_sim.null_service:Launching Console at port 52237
Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


#### Create the wallets

In [6]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

#### Create the settlment asset and mint

In [7]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

In [8]:
tdai_id = vega.find_asset_id(symbol="tDAI")

In [9]:
vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

#### Update network parameters we want 

In [10]:
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.stakeToCcySiskas", "1.0")
vega.forward('10s')
vega.wait_for_total_catchup()
vega.update_network_parameter(MM_WALLET.name, "market.liquidity.probabilityOfTrading.tau.scaling", "10.0")

#### Create the market and set up liquidity

In [11]:
mdp=1
tick=np.power(10.0,-mdp)
vega.create_simple_market(
        market_name="ETHUSD_Dec23",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=mdp,
        risk_aversion=0.000001,
        tau=1.0/365.25/24,
        sigma=1,
    )

In [12]:
market_id = vega.all_markets()[0].id
print(market_id)

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


5b1492fdf194aa5c5b7b76014b61a48dafe1660e4b53f24267eb38e01ca19379


In [13]:
vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=600,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_MID",
    reference_sell="PEGGED_REFERENCE_MID",
    delta_buy=10,
    delta_sell=10,
    is_amendment=False,
)

vega.forward('1s')
vega.forward('1s')

In [14]:
# Submit orders which will stay on the book and create best bid / ask 
price = 1500
delta_minus = 5
delta_plus = 5
vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price-delta_minus,
    wait=False,
)

vega.submit_order(
    trading_wallet=MM_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price+delta_plus,
    wait=False,
)



In [15]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 99999204.17474
mm Margin  = 195.82526
mm Bond    = 600.0
T1 General = 1000000.0
T1 Margin  = 0
T1 Bond    = 0
T2 General = 1000000.0
T2 Margin  = 0
T2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


#### Do some trading

In [16]:
# Do a trade which will cross
vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=price,
    wait=False,
)

vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=price,
    wait=False,
)
vega.forward('10s')

# We will now have mark price of 'price' 


In [17]:
num_levels = 20
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print("ask prices / volumes")
print(ask_prices)
print(ask_volumes)

print("bid prices / volumes")
print(bid_prices)
print(bid_volumes)


ask prices / volumes
[1505.0, 1510.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1.0, 0.87, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
bid prices / volumes
[1495.0, 1490.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1.0, 0.88, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

mm General = 99999217.51167
mm Margin  = 182.48833
mm Bond    = 600.0
T1 General = 999901.24344
T1 Margin  = 98.75656
T1 Bond    = 0
T2 General = 999896.41266
T2 Margin  = 103.58734
T2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


In [19]:
from math import ceil, floor

percent_bound = 0.02
best_ask = price + delta_plus
num_ask_ticks = ceil(((1+percent_bound)*price-best_ask) / tick)
print("number of ask ticks "+str(num_ask_ticks))

best_bid = price - delta_minus
num_bid_ticks = ceil((best_bid-(1-percent_bound)*price )/tick)
print("number of bid ticks "+str(num_bid_ticks))

vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=60000,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=(num_bid_ticks+1)*tick,
    delta_sell=(num_ask_ticks+1)*tick,
    is_amendment=True,
)



number of ask ticks 250
number of bid ticks 250


In [20]:
vega.wait_for_total_catchup()
num_levels = 20
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print("ask prices / volumes")
print(ask_prices)
print(ask_volumes)

print("bid prices / volumes")
print(bid_prices)
print(bid_volumes)



for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

    # positions = vega.positions_by_market(wallet_name=wallet.name, market_id=market_id)
    # print(positions)


ask prices / volumes
[1505.0, 1530.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1.0, 126.33, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
bid prices / volumes
[1495.0, 1469.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1.0, 131.73, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
mm General = 99927574.20409
mm Margin  = 12425.79591
mm Bond    = 60000.0
T1 General = 999901.24344
T1 Margin  = 98.75656
T1 Bond    = 0
T2 General = 999896.41266
T2 Margin  = 103.58734
T2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


In [21]:
# Now I want 1 mil of notional orders within say 2% on each side

start_vol=1
vol_step=1 
notional_remaining = 1e6 
while notional_remaining >= 0:
    for i in range(0,num_ask_ticks):
        vega.submit_order(
            trading_wallet=MM_WALLET.name,
            market_id=market_id,
            order_type="TYPE_LIMIT",
            time_in_force="TIME_IN_FORCE_GTC",
            side="SIDE_SELL",
            volume=vol_step*i,
            price=price+delta_plus+i*tick,
            wait=False,
        )
        notional_remaining = notional_remaining - (price+delta_plus+i*tick)*vol_step*i
        if notional_remaining <= 0:
            break 
    print("notional remaining " + str(notional_remaining))

notional_remaining = 1e6 
while notional_remaining >= 0:
    for i in range(0,num_ask_ticks):
        vega.submit_order(
            trading_wallet=MM_WALLET.name,
            market_id=market_id,
            order_type="TYPE_LIMIT",
            time_in_force="TIME_IN_FORCE_GTC",
            side="SIDE_BUY",
            volume=vol_step*i,
            price=price-delta_minus-i*tick,
            wait=False,
        )
        notional_remaining = notional_remaining - (price-delta_minus-i*tick)*vol_step*i
        if notional_remaining <= 0:
            break 
    print("notional remaining " + str(notional_remaining))

notional remaining -3950.5999999999985
notional remaining -49227.5


In [22]:
num_levels = 20
book_state = vega.market_depth(
            market_id, num_levels=num_levels
        )
bid_prices=[level.price for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_prices=[level.price for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))
bid_volumes=[level.volume for level in book_state.buys] + [0] * max(0, num_levels - len(book_state.buys))
ask_volumes=[level.volume for level in book_state.sells] + [0] * max(0, num_levels - len(book_state.sells))

print("ask prices / volumes")
print(ask_prices)
print(ask_volumes)

print("bid prices / volumes")
print(bid_prices)
print(bid_volumes)


ask prices / volumes
[1505.0, 1505.1, 1505.2, 1505.3, 1505.4, 1505.5, 1505.6, 1505.7, 1505.8, 1505.9, 1506.0, 1506.1, 1506.2, 1506.3, 1506.4, 1506.5, 1506.6, 1506.7, 1506.8, 1506.9]
[1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]
bid prices / volumes
[1495.0, 1494.9, 1494.8, 1494.7, 1494.6, 1494.5, 1494.4, 1494.3, 1494.2, 1494.1, 1494.0, 1493.9, 1493.8, 1493.7, 1493.6, 1493.5, 1493.4, 1493.3, 1493.2, 1493.1]
[1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]


In [23]:
vega.wait_for_total_catchup()

for wallet in wallets:
    general, margin, bond = vega.party_account(wallet_name=wallet.name, asset_id=tdai_id, market_id=market_id)
    print(wallet.name + " General = "+str(general))
    print(wallet.name + " Margin  = "+str(margin))
    print(wallet.name + " Bond    = "+str(bond))

    # positions = vega.positions_by_market(wallet_name=wallet.name, market_id=market_id)
    # print(positions)


mm General = 99874699.3835
mm Margin  = 65300.6165
mm Bond    = 60000.0
T1 General = 999901.24344
T1 Margin  = 98.75656
T1 Bond    = 0
T2 General = 999896.41266
T2 Margin  = 103.58734
T2 Bond    = 0
Settle General = 0
Settle Margin  = 0
Settle Bond    = 0


In [64]:
vega.settle_market(settlement_wallet = TERMINATE_WALLET.name, settlement_price = price, market_id=market_id)

In [1]:
vega.stop()

NameError: name 'vega' is not defined